<a href="https://colab.research.google.com/github/Guljeren/Ml_lab/blob/main/lab_2_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def parse_lenta_news(num_news=10):
    """Парсит указанное количество новостей с Lenta.ru"""
    url = "https://lenta.ru/parts/news/"
    r = requests.get(url)
    if r.status_code != 200: return []

    soup = BeautifulSoup(r.text, 'html.parser')
    news_items = soup.find_all('a', class_='card-full-news _parts-news')

    # Берем только указанное количество новостей
    available = len(news_items)
    num_news = min(num_news, available)
    news_items = news_items[:num_news]

    news_list = []
    for item in news_items:
        title = item.get_text(strip=True)
        link = item.get('href')
        if link and link.startswith('/'):
            link = 'https://lenta.ru' + link

        time_tag = item.find('time')
        time_text = time_tag.get_text(strip=True) if time_tag else 'Нет времени'

        # Получаем полный текст новости
        full_text = "Текст не получен"
        if link:
            try:
                article_r = requests.get(link, timeout=5)
                if article_r.status_code == 200:
                    article_soup = BeautifulSoup(article_r.text, 'html.parser')
                    text_block = article_soup.find('div', class_='topic-body__content')
                    if text_block:
                        paragraphs = text_block.find_all('p')
                        full_text = ' '.join([p.get_text(strip=True) for p in paragraphs])
            except:
                pass

        news_list.append({'time': time_text, 'title': title, 'link': link, 'full_text': full_text})

    return news_list

# Основной блок
num_news = 5  # Количество новостей для парсинга
news_data = parse_lenta_news(num_news)

print(f"Успешно собрано: {len(news_data)} новостей")
if news_data:
    df = pd.DataFrame(news_data)
    print(f"\nDataFrame размер: {df.shape}")
    print(df[['time', 'title']].head())
    df.to_csv('lenta_news.csv', index=False, encoding='utf-8-sig')

Успешно собрано: 5 новостей

DataFrame размер: (5, 4)
    time  \
0  04:24   
1  04:23   
2  04:22   
3  04:11   
4  04:00   

                                                                                                 title  
0  «Спасибо за координаты!» ГУР и ВСУ устроили перестрелку в элитном районе Киева и поплатились04:2...  
1                                                                  Трамп получит Премию мира04:23Спорт  
2                                 В США откровенно высказались о положении Украины в конфликте04:22Мир  
3                                   Вэнс назвал конфликт на Украине «источником разочарования»04:11Мир  
4              «Слишком маленький» пенис студента оказался размером с гигантскую колбаску04:00Из жизни  
